In [1]:
import pennylane as qml
import numpy as np
import time
from tqdm import tqdm

import quimb.tensor as qtn

import sparse_circuit as sc

from sparse_circuit import haar_2qbit_square, zero_bitstring_coo
from sparse_circuit import construct_random_wire_list, convert_gate_to_coo, trim_n_coo, construct_worst_wire_list
from sparse_circuit import quantum_circuit
from sparse_circuit import compute_mps_metrics
from sparse_circuit import load_run_data, save_run_data

# Run with a constant qubit number

### 16 Qubits

In [ ]:
n_qbits = 16
n_layers = 5
n_vals = 20

min_dim = 2**(n_qbits/2)/16
max_dim = 2**(n_qbits/2)

compute_mps_metrics(n_qbits, n_layers, min_dim, max_dim, n_vals, layer_type='random')

### 18 Qubits

In [ ]:
n_qbits = 18
n_layers = 5
n_vals = 20

min_dim = 2**(n_qbits/2)/16
max_dim = 2**(n_qbits/2)

compute_mps_metrics(n_qbits, n_layers, min_dim, max_dim, n_vals, layer_type='worst')

### 20 Qubits

In [ ]:
n_qbits = 20
n_layers = 5
n_vals = 20

min_dim = 2**(n_qbits/2)/16
max_dim = 2**(n_qbits/2)

compute_mps_metrics(n_qbits, n_layers, min_dim, max_dim, n_vals, layer_type='worst')

### 22 Qubits

In [ ]:
n_qbits = 22
n_layers = 5
n_vals = 20

min_dim = 2**(n_qbits/2)/16
max_dim = 2**(n_qbits/2)

compute_mps_metrics(n_qbits, n_layers, min_dim, max_dim, n_vals, layer_type='worst')

### 24 Qubits

In [ ]:
n_qbits = 24
n_layers = 5
n_vals = 20

min_dim = 2**(n_qbits/2)/16
max_dim = 2**(n_qbits/2)

compute_mps_metrics(n_qbits, n_layers, min_dim, max_dim, n_vals, layer_type='worst')

# Increase qubit count

### 16 Bond Dim

In [ ]:
n_qbits_list = np.arange(12, 32)
n_layers = 5

#we use the same max bond dimension for every 
max_bond_dim = int(2**(n_qbits_list[0]/2)/4)

fidelities = np.zeros(len(n_qbits_list))
times = np.zeros_like(fidelities)

# start_time=time.time()
for i in tqdm(range(len(n_qbits_list))):
    #define gates and wires
    wires_list = np.concat([sc.construct_worst_wire_list(n_qbits_list[i]) for _ in range(n_layers)])
    gates_list = [sc.haar_2qbit_square() for _ in wires_list]
    gates_list_quimb = [qtn.Gate.from_raw(g, w) for g,w  in zip(gates_list, wires_list)]

    #define input
    in_vals = tuple(0 for _ in range(n_qbits_list[i]))

    #exact circuit output
    circ_exact = qtn.Circuit(n_qbits_list[i])
    circ_exact.apply_gates(gates_list_quimb)
    output_exact = circ_exact.psi.contract()

    t0=time.time()
    #approximate circuit output
    circ_approx = qtn.CircuitPermMPS(n_qbits_list[i], max_bond=max_bond_dim)
    t0 = time.time()
    circ_approx.apply_gates(gates_list_quimb)
    output_approx = circ_approx.psi

    times[i] = time.time()-t0
    fidelities[i] = np.abs(output_approx.conj() @ output_exact)**2

save_run_data(f'results/increase_qubit_count_{int(max_bond_dim)}.pkl', n_layers=n_layers, n_qbits=n_qbits_list, fidelity=fidelities, time_taken=times, max_bond_dim=max_bond_dim)

  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_3810/1992859811.py:33: ComplexWarning: Casting complex values to real discards the imaginary part
  fidelities[i] = (output_approx.conj() @ output_exact)**2
100%|██████████| 10/10 [00:00<00:00, 24.49it/s]


### 32 Max bond Dim

In [ ]:
n_qbits_list = np.arange(12, 32)
n_layers = 5

#we use the same max bond dimension for every 
max_bond_dim = int(2**(n_qbits_list[0]/2)/2)

fidelities = np.zeros(len(n_qbits_list))
times = np.zeros_like(fidelities)

for i in tqdm(range(len(n_qbits_list))):
    #define gates and wires
    wires_list = np.concat([sc.construct_worst_wire_list(n_qbits_list[i]) for _ in range(n_layers)])
    gates_list = [sc.haar_2qbit_square() for _ in wires_list]
    gates_list_quimb = [qtn.Gate.from_raw(g, w) for g,w  in zip(gates_list, wires_list)]

    #define input
    in_vals = tuple(0 for _ in range(n_qbits_list[i]))

    #exact circuit output
    circ_exact = qtn.Circuit(n_qbits_list[i])
    circ_exact.apply_gates(gates_list_quimb)
    output_exact = circ_exact.psi.contract()

    t0=time.time()
    #approximate circuit output
    circ_approx = qtn.CircuitPermMPS(n_qbits_list[i], max_bond=max_bond_dim)
    t0 = time.time()
    circ_approx.apply_gates(gates_list_quimb)
    output_approx = circ_approx.psi

    times[i] = time.time()-t0
    fidelities[i] = np.abs(output_approx.conj() @ output_exact)**2

save_run_data(f'results/increase_qubit_count_{int(max_bond_dim)}.pkl', n_layers=n_layers, n_qbits=n_qbits_list, fidelity=fidelities, time_taken=times, max_bond_dim=max_bond_dim)

### 64 max bond dim

In [ ]:
n_qbits_list = np.arange(12, 32)
n_layers = 5

#we use the same max bond dimension for every 
max_bond_dim = int(2**(n_qbits_list[0]/2)/1)

fidelities = np.zeros(len(n_qbits_list))
times = np.zeros_like(fidelities)

for i in tqdm(range(len(n_qbits_list))):
    #define gates and wires
    wires_list = np.concat([sc.construct_worst_wire_list(n_qbits_list[i]) for _ in range(n_layers)])
    gates_list = [sc.haar_2qbit_square() for _ in wires_list]
    gates_list_quimb = [qtn.Gate.from_raw(g, w) for g,w  in zip(gates_list, wires_list)]

    #define input
    in_vals = tuple(0 for _ in range(n_qbits_list[i]))

    #exact circuit output
    circ_exact = qtn.Circuit(n_qbits_list[i])
    circ_exact.apply_gates(gates_list_quimb)
    output_exact = circ_exact.psi.contract()

    t0=time.time()
    #approximate circuit output
    circ_approx = qtn.CircuitPermMPS(n_qbits_list[i], max_bond=max_bond_dim)
    t0 = time.time()
    circ_approx.apply_gates(gates_list_quimb)
    output_approx = circ_approx.psi

    times[i] = time.time()-t0
    fidelities[i] = np.abs(output_approx.conj() @ output_exact)**2

save_run_data(f'results/increase_qubit_count_{int(max_bond_dim)}.pkl', n_layers=n_layers, n_qbits=n_qbits_list, fidelity=fidelities, time_taken=times, max_bond_dim=max_bond_dim)